In [65]:
import intake
import intake_xarray

import numpy as np
import xarray as xr

import rioxarray #noqa


from pathlib import Path
from functools import partial

In [66]:
cat = intake.entry.Catalog()

In [67]:
pre = "../../../data/courses/"
f_gtc = "microwave-remote-sensing/gtc.zarr"
f_coef = "microwave-remote-sensing/coef.zarr"
f_rtc = "microwave-remote-sensing/rtc.zarr"

In [68]:
def _preprocess(x, xlims, ylims, scale):
    '''
    Preprocess file.

    Parameters
    ----------
    x : xarray.Dataset
    xlims: tuple
    ylims: tuple

    Returns
    -------
    xarray.Dataset
    '''
    x = x * scale
    x = x.sel(x=slice(*xlims), y=slice(*ylims))
    x = x.assign_coords(band=[Path(x.encoding["source"]).parent.stem])
    return x

In [69]:
data_path = Path("~/shared/datasets/rs/sentinel-1").expanduser()

# longitude and latitude
xlims, ylims = (9.5, 10), (47.5, 47)
partial_func = partial(_preprocess, xlims=xlims, ylims=ylims, scale=1)

gtc_dc = xr.open_mfdataset(
    (data_path / "gtc").glob("**/*.tif"),
    engine="rasterio",
    combine="nested",
    concat_dim="band",
    preprocess=partial_func,
    join="override",
    chunks={"x":1000, "y":1000},
    parallel=True
    ).\
    compute()

coef_dc = xr.open_mfdataset(
    (data_path / "coefs").glob("**/*.tif"),
    engine="rasterio",
    combine="nested",
    concat_dim="band",
    preprocess=partial_func,
    join="override",
    chunks={"x":1000, "y":1000},
    parallel=True
    ).\
    compute()

rtc_dc = xr.open_mfdataset(
    (data_path / "rtc").glob("**/*.tif"),
    engine="rasterio",
    combine="nested",
    concat_dim="band",
    preprocess=partial_func,
    join="override",
    chunks={"x":1000, "y":1000},
    parallel=True
    ).\
    compute()

In [70]:
gtc_dc.to_zarr(f_gtc, mode="w")
coef_dc.to_zarr(f_coef, mode="w")
rtc_dc.to_zarr(f_rtc, mode="w")

In [71]:
for i in [f_gtc, f_coef, f_rtc]:
    source = intake.open_zarr(
        i,
        decode_coords="all",
        chunks={},
    )
    cat[str(Path(i).stem)] = source.reader()

In [72]:
cat.to_yaml_file("microwave-remote-sensing.yml")